# Quantum melodies played on simulated guitar strings

In [12]:
import numpy as np
import numba
from numba import jit

%store -r circuits_melodies

In [88]:
#generating music for chosen circuit in range 1 to 12

# Getting keyboard input from the user for the circuit number
circuit_no = input("Enter the circuit number (1 to 12): ")
# Converting the input to an integer
circuit_no = int(circuit_no)
quant_frequencies = circuits_melodies[circuit_no-1]
durations_base = np.full(len(quant_frequencies), 0.15)


print("Quant frequencies:", quant_frequencies)


Enter the circuit number (1 to 12):  12


Quant frequencies: [275.10570052 184.35050767 306.17885317 337.38019506 368.6627692
 400.         244.2519199  213.81942498 157.79600718]


## Composing counterpoints

### Contrary motion mode

The principal of contrary motion is a well known composing techique based on mirroring the melodic line in a second voice. Each leap in the first voice is mirrored by a simultaneous leap in the other oposite direction in the second voice. 

In [89]:
ctr_mtn_freq = []

for i in range(len(quant_frequencies)):
    if i == 0:
        ctr_mtn_freq.append(quant_frequencies[i])
    else:
        value = quant_frequencies[i] - quant_frequencies[i-1]
        ctr_mtn_freq.append(ctr_mtn_freq[i-1] - value)
    

## Creating .wav files

### Guiar string simulation

Guitar string length  $\boxed{L = 0.7~\text{m}}$

Choose $\boxed{N_x=101}$ guitar string positions $\implies$ $\boxed{\Delta x = 0.7~\text{mm}}$

Note that the fundamental frequency of a guitar note is $f = c/2L$. With an "A note" at 220Hz get $\boxed{c = 308~\text{m/s}} $

To obey our constraint we thus set $\boxed{\Delta t = 5 \times 10^{-6} s}$. In order to get multiple seconds of a result, choose $\boxed{N_t = 500000}$

Two parameters that seemed to give a solution that sounded like a string were $\boxed{l= 2 \times 10^{-6}}$ and $\boxed{\gamma = 2.6 \times 10^{-5} s/m}$



In [16]:
#creating an empty list to hold the solutions for each frequency
def generate(durations, frequencies):
    arr_solution = []

    Nx = 101
    L = 0.7
    dx = L / (Nx - 1)
    dt = 5e-6
    l = 5e-5
    gamma = 5e-5


    for i in range(len(frequencies)):

        
        base_freq = frequencies[i] #the base frequency
        if base_freq == 0:
            base_freq = 200
        c = 2 * L * base_freq
        
        
        desired_duration = durations[i]
        #calculating the corresponding number of time steps (Nt) based on the desired duration
        Nt = int(desired_duration / dt)
        

        #initial state of a guitar string:
        ya = np.linspace(0, 0.01, 70)
        yb = np.linspace(0.01, 0, 31)
        y0 = np.concatenate([ya, yb])

        #creating 2D array of 𝑦(𝑥,𝑡)
        sol = np.zeros((Nt, Nx))

        #making the solution at 𝑡=0 and 𝑡=1 equal to the guitar "pluck"
        sol[0] = y0
        sol[1] = y0

        #going through the iterative procedure:
        @numba.jit("f8[:,:](f8[:,:], i8, i8, f8, f8, f8, f8)", nopython=True, nogil=True)
        def compute_d(d, times, length, dt, dx, l, gamma):
            for t in range(1, times - 1):
                for i in range(2, length - 2):
                    outer_fact = (1 / (c ** 2 * dt ** 2) + gamma / (2 * dt)) ** (-1)
                    p1 = 1 / dx ** 2 * (d[t][i - 1] - 2 * d[t][i] + d[t][i + 1])
                    p2 = 1 / (c ** 2 * dt ** 2) * (d[t - 1][i] - 2 * d[t][i])
                    p3 = gamma / (2 * dt) * d[t - 1][i]
                    p4 = l ** 2 / dx ** 4 * (d[t][i + 2] - 4 * d[t][i + 1] + 6 * d[t][i] - 4 * d[t][i - 1] + d[t][i - 2])
                    d[t + 1][i] = outer_fact * (p1 - p2 + p3 - p4)
            return d

        sol = compute_d(sol, Nt, Nx, dt, dx, l, gamma)

        #appending the solution for the current frequency to the list
        arr_solution.append(sol)
        
    return arr_solution

   


Extracting the "amount" of the harmonics at any time $t$:

$$ \text{Amplitude of harmonic n at time t} \propto \int_{0}^L y(x, t) \sin(n \pi x / L) dx $$

In [17]:
def compose_melody(arr_solution):

    #melody :)
    audio_data = []

    for i in range(len(arr_solution)):
        #adding harmonics together
        tot = arr_solution[i].sum(axis=1)[::10] # all harmonics
        tot = tot.astype(np.float32)

        #appending the audio data to the list
        audio_data.append(tot)
    
    return audio_data

### Making a WAV file of main melody

In [18]:
from scipy.io import wavfile
from IPython.display import Audio

In [90]:
arr_solution_base = generate(durations_base,quant_frequencies)
base_melody = compose_melody(arr_solution_base)
# Combine the audio data from different frequencies into one array
combined_audio_base = np.concatenate(base_melody)
file_name_base = "base_melody"+str(circuit_no)+".wav"
wavfile.write(file_name_base,20000,combined_audio_base)

In [58]:
Audio(file_name_base)

### Making a WAV file of second voice moving in contary motion

In [91]:
arr_solution_contr = generate(durations_base,ctr_mtn_freq)
contr_melody = compose_melody(arr_solution_contr)
# Combine the audio data from different frequencies into one array
combined_audio_contr = np.concatenate(contr_melody)
file_name_contr = "contr_melody"+str(circuit_no)+".wav"
wavfile.write(file_name_contr,20000,combined_audio_contr)

In [32]:
Audio(file_name_contr)

Cobining the two voices into one wav file

In [55]:
!pip install pydub


[notice] A new release of pip available: 23.1.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [92]:
from pydub import AudioSegment

# Load the first audio track
track1 = AudioSegment.from_file(file_name_contr, format="wav")

# Load the second audio track
track2 = AudioSegment.from_file(file_name_base, format="wav")

# Overlay the second track on top of the first track at position 0ms
overlayed = track1.overlay(track2, position=0)

# Export the overlayed audio as a WAV file
final_file_name_contrary_motion = "final_samples/contrary_motion_" + str(circuit_no) + ".wav"
overlayed.export(final_file_name_contrary_motion, format="wav")



<_io.BufferedRandom name='final_samples/contrary_motion_12.wav'>

In [78]:
Audio(final_file_name_contrary_motion)

### Composing wav for phase shift mode

Phase shifting as a composition technique is a musical practice used by minimalist composers. Phase shifting brings a simple melody to life, adding rythmical and harmonical complexity.

In [93]:
# generating the base melody once again in 0.5 speed because the phase shifting will cause acceleration

durations_base = np.full(len(quant_frequencies), 0.3)
arr_solution_base = generate(durations_base,quant_frequencies)
base_melody = compose_melody(arr_solution_base)
# Combine the audio data from different frequencies into one array
combined_audio_base = np.concatenate(base_melody)
file_name_base = "base_melody"+str(circuit_no)+".wav"
wavfile.write(file_name_base,20000,combined_audio_base)


In [94]:
# Load the first audio track
track1 = AudioSegment.from_file(file_name_base, format="wav")

# Load the second audio track
track2 = AudioSegment.from_file(file_name_base, format="wav")

# Overlay the second track on top of the first track at position 50ms
overlayed = track1.overlay(track2, position=750)

# Export the overlayed audio as a WAV file
final_file_name_phase_shift = "final_samples/phase_shift_" + str(circuit_no) + ".wav"
overlayed.export(final_file_name_phase_shift, format="wav")

<_io.BufferedRandom name='final_samples/phase_shift_12.wav'>

In [95]:
Audio(final_file_name_phase_shift)